데이터 설명 : e-commerce 배송의 정시 도착여부 (1: 정시배송 0 : 정시미배송)

In [1]:
import pandas as pd
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv")


In [2]:
print(x_train.info())
print(x_test.info())

display(x_train.head())
display(y_train.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6598 entries, 0 to 6597
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   6598 non-null   int64 
 1   Warehouse_block      6598 non-null   object
 2   Mode_of_Shipment     6598 non-null   object
 3   Customer_care_calls  6598 non-null   object
 4   Customer_rating      6598 non-null   int64 
 5   Cost_of_the_Product  6598 non-null   int64 
 6   Prior_purchases      6598 non-null   int64 
 7   Product_importance   6598 non-null   object
 8   Gender               6598 non-null   object
 9   Discount_offered     6598 non-null   int64 
 10  Weight_in_gms        6598 non-null   int64 
dtypes: int64(6), object(5)
memory usage: 567.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4401 entries, 0 to 4400
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  -

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,6045,A,Flight,4,3,266,5,high,F,5,1590
1,44,F,Ship,3,1,174,2,low,M,44,1556
2,7940,F,Road,4,1,154,10,high,M,10,5674
3,1596,F,Ship,4,3,158,3,medium,F,27,1207
4,4395,A,Flight,5,3,175,3,low,M,7,4833


,ID,Reached.on.Time_Y.N
0,6045,0
1,44,1
2,7940,1
3,1596,1
4,4395,1


In [3]:
print(x_train.isna().sum())
print(x_test.isna().sum())

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64
ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64


In [4]:
print(x_train.nunique())

x_train_drop = x_train.drop(columns = ['ID'])
x_test_drop = x_test.drop(columns = ['ID'])

ID                     6598
Warehouse_block           5
Mode_of_Shipment          3
Customer_care_calls       6
Customer_rating           5
Cost_of_the_Product     215
Prior_purchases           8
Product_importance        3
Gender                    2
Discount_offered         65
Weight_in_gms          3365
dtype: int64


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

x_train_dum = pd.get_dummies(x_train_drop)
x_test_dum = pd.get_dummies(x_test_drop)
x_test_dum = x_test_dum[x_train_dum.columns]
y = y_train['Reached.on.Time_Y.N']

In [6]:
x_train, x_val, y_train, y_val = train_test_split(x_train_dum, y, test_size=0.3, random_state = 23)
rf = RandomForestClassifier(random_state=23)
rf.fit(x_train, y_train)


RandomForestClassifier(random_state=23)

In [7]:

from sklearn.metrics import accuracy_score , f1_score, recall_score, roc_auc_score ,precision_score

# 학습 데이터
predict_train_label = rf.predict(x_train) #예측값 계산
predict_train_proba = rf.predict_proba(x_train)[:,1]#확률값 계산, [클래스0: 이탈x, 클래스1: 이탈o] 형식
#(클래스 1에 대한 확률값 -> 찾고자하는 이탈값)

# 검증 데이터
predict_validation_label = rf.predict(x_val)#예측값 계산
predict_validation_prob = rf.predict_proba(x_val)[:,1]#확률값 계산

print('train accuracy :', accuracy_score(y_train,predict_train_label))
print('validation accuracy :', accuracy_score(y_val,predict_validation_label))
print('\n')
print('train f1_score :', f1_score(y_train,predict_train_label))
print('validation f1_score :', f1_score(y_val,predict_validation_label))
print('\n')
print('train recall_score :', recall_score(y_train,predict_train_label))
print('validation recall_score :', recall_score(y_val,predict_validation_label))
print('\n')
print('train precision_score :', precision_score(y_train,predict_train_label))
print('validation precision_score :', precision_score(y_val,predict_validation_label))
print('\n')
print('train auc :', roc_auc_score(y_train,predict_train_proba))
print('validation auc :', roc_auc_score(y_val,predict_validation_prob))



train accuracy : 1.0
validation accuracy : 0.6545454545454545


train f1_score : 1.0
validation f1_score : 0.6904977375565611


train recall_score : 1.0
validation recall_score : 0.6510238907849829


train precision_score : 1.0
validation precision_score : 0.7350674373795761


train auc : 1.0
validation auc : 0.7362335476295071


In [8]:
# 답안제출
predict_test_label = rf.predict(x_test_dum)
predict_test_proba = rf.predict_proba(x_test_dum)[:,1]

pd.DataFrame({
    'ID': x_test.ID, 
    'Reached.on.Time_Y.N': predict_test_label
}).to_csv('result.csv', index=False)
